<a href="https://colab.research.google.com/github/curlos/manga-panel-splitter/blob/main/Magi_V2_Model_Analyze_Images_(Manga_Panel_Splitter).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==4.40 pulp pyngrok python-dotenv accelerate

In [ ]:
!pkill -f ngrok

In [ ]:
import requests
from flask import Flask, request, jsonify
import os
import numpy as np
from transformers import AutoModel
import torch
from pyngrok import ngrok
import threading
import base64
import pdb
from google.colab import drive
from dotenv import load_dotenv

# Mount Google Drive if it hasn't mounted already.
if not os.path.ismount('/content/drive'):
    drive.mount('/content/drive')

# Path to the .env file in your Drive
env_path = '/content/drive/MyDrive/google_colab_env_files/.env'

# Load the .env file
load_dotenv(env_path)

# Access the ngrok auth token
ngrok_auth_token = os.getenv("NGROK_AUTH_TOKEN")

if not ngrok_auth_token:
    raise ValueError("NGROK_AUTH_TOKEN is not set. Check your .env file in Google Drive ('google_colab_env_files/.env').")

ngrok.set_auth_token(ngrok_auth_token)


app = Flask(__name__)
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # Increase limit to 16MB

# Initialize the Magi model
magi_model = AutoModel.from_pretrained(
            "ragavsachdeva/magiv2", trust_remote_code=True).eval()

def get_per_page_results(magi_model, chapter_pages, character_bank):
  # Set to "no_grad()" so that there's inference without tracking gradients. Basically, this saves memory and computational resources by turning off gradient tracking.
  with torch.no_grad():
      per_page_results = magi_model.do_chapter_wide_prediction(
          chapter_pages, character_bank, use_tqdm=True, do_ocr=True
      )

  return per_page_results

@app.route('/process-images-with-magi-model', methods=['POST'])
def process_images_with_magi_model():
    try:
      # Parse JSON payload
      request_data = request.json
      encoded_arrays = request_data.get("chapter_pages_image_numpy_array")
      character_bank = request_data.get("character_bank")

      pdb.set_trace()

      # Decode and reconstruct the arrays
      chapter_pages_image_numpy_array = [
          np.frombuffer(base64.b64decode(item["data"]), dtype=item["dtype"]).reshape(item["shape"])
          for item in encoded_arrays
      ]

      # Run Magi model on the file
      per_page_results = get_per_page_results(
          magi_model, chapter_pages_image_numpy_array, character_bank
      )

      pdb.set_trace()

      return per_page_results
    except Exception as e:
      return jsonify({"error": str(e)}), 500

@app.route('/hello-world', methods=['GET'])
def hello_world():
    return 'Hello World!'

    # Send back the processed numpy array
    # return jsonify({"processed_array": processed_array.tolist()})

def run_flask():
  # Run Flask app without reloader (important for threading)
  app.run(host="0.0.0.0", port=8893, debug=True, use_reloader=False)

if __name__ == '__main__':
    # Start Flask in a thread to prevent blocking
    thread = threading.Thread(target=run_flask)
    thread.start()

    # Expose the Flask app to the internet using ngrok
    public_url = ngrok.connect(8893)
    print("Public URL:", public_url)


2qwDNm81vHgbdQn94RYAiyt2R2V_6JUmhkXmm4ZzbyqwBcBu1
Magiv2Model(
  (ocr_model): VisionEncoderDecoderModel(
    (encoder): ViTModel(
      (embeddings): ViTEmbeddings(
        (patch_embeddings): ViTPatchEmbeddings(
          (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
        )
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (encoder): ViTEncoder(
        (layer): ModuleList(
          (0-11): 12 x ViTLayer(
            (attention): ViTAttention(
              (attention): ViTSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=False)
                (key): Linear(in_features=768, out_features=768, bias=False)
                (value): Linear(in_features=768, out_features=768, bias=False)
                (dropout): Dropout(p=0.0, inplace=False)
              )
              (output): ViTSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.0, 

Address already in use
Port 8893 is in use by another program. Either identify and stop that program, or start the server with a different port.


Public URL: NgrokTunnel: "https://8a94-35-198-207-72.ngrok-free.app" -> "http://localhost:8893"
